# now do some ML using the web scraped data

In [653]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [654]:
from player_data_expansion import get_all_expanded_player_fight_data

In [655]:
# AWS integration getting file data
df_init = pd.read_csv("fight_data.csv").fillna(pd.NA)
df_init.head()

,Date,fighter1,fighter1_result,method_or_round,fighter2,Venue,Undercard fights,link,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,"Saturday, 3 May 2025",Canelo Alvarez,Winner,"UD, Round 12",William Scull,"The Venue, Riyadh, Saudi Arabia",Jaime Munguia Beat Bruno Surace (UD 12) \nMar...,https://box.live/fights/canelo-vs-scull/,62,2,2,23,0,0,175,182.0,0,0
1,"Saturday, 3 May 2025",Jukiya Iimura,Winner,"UD, Round 12",Esneth Domingo,"Korakuen Hall, Tokyo, Japan",<NA>,https://box.live/fights/iimura-vs-domingo/,8,1,0,21,2,0,164,160.0,0,0
2,"Friday, 2 May 2025",Ryan Garcia,Loser,"UD, Round 12",Rolando ‘Rolly’ Romero,"Times Square, New York, NY, United States",Devin Haney Beat Jose Ramirez (UD 12) \nTeofi...,https://box.live/fights/garcia-vs-romero/,24,1,0,16,2,0,178,173.0,83,81
3,"Thursday, 1 May 2025",Kevin Lerena,Winner,"TKO, Round 3",Serhiy Radchenko,"Sunbet Arena, Pretoria, South Africa",<NA>,https://box.live/fights/lerena-vs-radchenko/,30,3,0,11,7,0,185,186.0,47,45
4,"Sunday, 27 April 2025",Marlon Tapales,Winner,"RTD, Round 3",Jon Jon Jet,"General Santos City, Cotabato del Sur, Philipp...",<NA>,https://box.live/fights/tapales-vs-jet/,39,4,0,15,1,1,163,166.0,0,0


### start cleaning

In [656]:
df_essentials = df_init.drop(columns=['link','Venue','Date','Undercard fights','fighter1','fighter2'])
df_essentials.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"UD, Round 12",62,2,2,23,0,0,175,182.0,0,0
1,Winner,"UD, Round 12",8,1,0,21,2,0,164,160.0,0,0


In [657]:
df_staged_cleaning = df_essentials.copy()
df_staged_cleaning.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"UD, Round 12",62,2,2,23,0,0,175,182.0,0,0
1,Winner,"UD, Round 12",8,1,0,21,2,0,164,160.0,0,0


In [658]:
df_staged_cleaning['method_or_round'] = df_staged_cleaning['method_or_round'].str.split(' ')

In [659]:
last_rounds = [new_list[2] for new_list in df_staged_cleaning['method_or_round']][:]
df_staged_cleaning['round'] = last_rounds

df_staged_cleaning.head()

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,"[UD,, Round, 12]",62,2,2,23,0,0,175,182.0,0,0,12
1,Winner,"[UD,, Round, 12]",8,1,0,21,2,0,164,160.0,0,0,12
2,Loser,"[UD,, Round, 12]",24,1,0,16,2,0,178,173.0,83,81,12
3,Winner,"[TKO,, Round, 3]",30,3,0,11,7,0,185,186.0,47,45,3
4,Winner,"[RTD,, Round, 3]",39,4,0,15,1,1,163,166.0,0,0,3


In [660]:
df_staged_cleaning = df_staged_cleaning.drop(columns='method_or_round')
df_staged_cleaning.head()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,62,2,2,23,0,0,175,182.0,0,0,12
1,Winner,8,1,0,21,2,0,164,160.0,0,0,12
2,Loser,24,1,0,16,2,0,178,173.0,83,81,12
3,Winner,30,3,0,11,7,0,185,186.0,47,45,3
4,Winner,39,4,0,15,1,1,163,166.0,0,0,3


### feature engineering swapped values + add to current values

In [661]:

df_staged_swapping = df_staged_cleaning.copy()

fighter2_wins_temp = df_staged_swapping['fighter2_record_wins'].copy()
fighter2_draws_temp = df_staged_swapping['fighter2_record_draws'].copy()
fighter2_losses_temp = df_staged_swapping['fighter2_record_losses'].copy()
fighter2_height_cm_temp = df_staged_swapping['fighter2_height_cm'].copy()
fighter2_ko_percentage_temp = df_staged_swapping['fighter2_ko_percentage'].copy()

df_staged_swapping['fighter2_record_wins'] = df_staged_swapping['fighter1_record_wins']
df_staged_swapping['fighter2_record_draws'] = df_staged_swapping['fighter1_record_draws']
df_staged_swapping['fighter2_record_losses'] = df_staged_swapping['fighter1_record_losses']
df_staged_swapping['fighter2_height_cm'] = df_staged_swapping['fighter1_height_cm']
df_staged_swapping['fighter2_ko_percentage'] = df_staged_swapping['fighter1_ko_percentage']

df_staged_swapping['fighter1_record_wins'] = fighter2_wins_temp
df_staged_swapping['fighter1_record_draws'] = fighter2_draws_temp
df_staged_swapping['fighter1_record_losses'] = fighter2_losses_temp
df_staged_swapping['fighter1_height_cm'] = fighter2_height_cm_temp
df_staged_swapping['fighter1_ko_percentage'] = fighter2_ko_percentage_temp

def reverse_results(rowval) : 
    if rowval == 'Winner' :
        return 'Loser'
    elif rowval == 'Loser' :
        return 'Winner'
    else :
        return 'Draw'
    
df_staged_swapping['fighter1_result'] = df_staged_swapping['fighter1_result'].apply(reverse_results)


concatenate the matrices

In [662]:
df_encoded = pd.concat([df_staged_cleaning, df_staged_swapping], axis=0)

In [663]:
df_encoded.head()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,62,2,2,23,0,0,175.0,182.0,0,0,12
1,Winner,8,1,0,21,2,0,164.0,160.0,0,0,12
2,Loser,24,1,0,16,2,0,178.0,173.0,83,81,12
3,Winner,30,3,0,11,7,0,185.0,186.0,47,45,3
4,Winner,39,4,0,15,1,1,163.0,166.0,0,0,3


could also feature engineer the ratio between fighters, such as fighter2/fighter1 for height

### encoding

In [664]:
# commented out so that the concatenation of swapped fighters is correctly set
# df_encoded = df_staged_cleaning.copy()

In [665]:
# WINNER = 0, LOSER = 1, DRAW = 2
def custom_encoder(value) :
    if value == 'Winner' :
        return 0
    elif value == 'Loser' :
        return 1
    else : return 2
df_encoded['fighter1_result'] = df_encoded['fighter1_result'].apply(custom_encoder)

In [666]:
df_encoded.shape

(24, 12)

In [667]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fighter1_result         24 non-null     int64  
 1   fighter1_record_wins    24 non-null     int64  
 2   fighter1_record_draws   24 non-null     int64  
 3   fighter1_record_losses  24 non-null     int64  
 4   fighter2_record_wins    24 non-null     int64  
 5   fighter2_record_draws   24 non-null     int64  
 6   fighter2_record_losses  24 non-null     int64  
 7   fighter1_height_cm      23 non-null     float64
 8   fighter2_height_cm      23 non-null     float64
 9   fighter1_ko_percentage  24 non-null     int64  
 10  fighter2_ko_percentage  24 non-null     int64  
 11  round                   24 non-null     object 
dtypes: float64(2), int64(9), object(1)
memory usage: 2.4+ KB


In [668]:
df_encoded = df_encoded.infer_objects()
df_encoded['fighter1_record_wins'] = df_encoded['fighter1_record_wins'].astype('Int64')
df_encoded['fighter1_record_draws'] = df_encoded['fighter1_record_draws'].astype('Int64')
df_encoded['fighter1_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_record_wins'] = df_encoded['fighter2_record_wins'].astype('Int64')
df_encoded['fighter2_record_draws'] = df_encoded['fighter2_record_draws'].astype('Int64')
df_encoded['fighter2_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_height_cm'] = df_encoded['fighter2_height_cm'].astype('Int64')
df_encoded['fighter1_ko_percentage'] = df_encoded['fighter1_ko_percentage'].astype('Int64')
df_encoded['fighter2_ko_percentage'] = df_encoded['fighter2_ko_percentage'].astype('Int64')
df_encoded['round'] = df_encoded['round'].astype('Int64')

In [669]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fighter1_result         24 non-null     int64  
 1   fighter1_record_wins    24 non-null     Int64  
 2   fighter1_record_draws   24 non-null     Int64  
 3   fighter1_record_losses  24 non-null     Int64  
 4   fighter2_record_wins    24 non-null     Int64  
 5   fighter2_record_draws   24 non-null     Int64  
 6   fighter2_record_losses  24 non-null     Int64  
 7   fighter1_height_cm      23 non-null     float64
 8   fighter2_height_cm      23 non-null     Int64  
 9   fighter1_ko_percentage  24 non-null     Int64  
 10  fighter2_ko_percentage  24 non-null     Int64  
 11  round                   24 non-null     Int64  
dtypes: Int64(10), float64(1), int64(1)
memory usage: 2.7 KB


In [670]:
df_encoded.describe()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
count,24.000000,24.0,24.0,24.0,24.0,24.0,24.0,23.000000,23.0,24.0,24.0,24.0
mean,0.625000,22.041667,2.125,0.333333,22.041667,2.125,0.333333,175.695652,175.695652,26.625,26.625,8.5
std,0.646899,12.052347,2.193122,0.56466,12.052347,2.193122,0.56466,8.503661,8.503661,33.657886,33.657886,3.878592
min,0.000000,6.0,0.0,0.0,6.0,0.0,0.0,160.000000,160.0,0.0,0.0,2.0
25%,0.000000,15.0,0.0,0.0,15.0,0.0,0.0,173.000000,173.0,0.0,0.0,6.0
50%,1.000000,21.5,2.0,0.0,21.5,2.0,0.0,175.000000,175.0,0.0,0.0,10.0
75%,1.000000,24.5,3.25,1.0,24.5,3.25,1.0,182.500000,182.5,59.5,59.5,12.0
max,2.000000,62.0,7.0,2.0,62.0,7.0,2.0,191.000000,191.0,83.0,83.0,12.0


### tt split

In [671]:
from sklearn.model_selection import train_test_split

In [672]:
X = df_encoded.drop(columns=['fighter1_result'])
y = df_encoded['fighter1_result']
y.unique()

array([0, 1, 2], dtype=int64)

In [673]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18, random_state=68) # ts .5 and rs 67
y_test.unique()

array([1, 0, 2], dtype=int64)

In [674]:
y_train

2     1
9     2
6     0
5     1
1     0
5     0
8     0
11    0
2     0
3     0
11    1
7     1
3     1
4     1
0     1
10    0
4     0
7     0
6     1
Name: fighter1_result, dtype: int64

## TRAIN MODEL - XGBoost First

In [675]:
# compute sample weight based on class frequency
class_weights = dict(enumerate(len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))))
sample_weight = np.array([class_weights[label] for label in y_train])
sample_weight

array([0.79166667, 6.33333333, 0.63333333, 0.79166667, 0.63333333,
       0.63333333, 0.63333333, 0.63333333, 0.63333333, 0.63333333,
       0.79166667, 0.79166667, 0.79166667, 0.79166667, 0.79166667,
       0.63333333, 0.63333333, 0.63333333, 0.79166667])

In [676]:
from xgboost import XGBClassifier

In [677]:
xgbc = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1
)

In [678]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [679]:
y_train = encoder.fit_transform(y_train)

In [680]:
xgbc.fit(X_train, y_train, sample_weight=sample_weight)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, ...)

In [681]:
xgbc_preds = xgbc.predict(X_test)
xgbc_preds = encoder.inverse_transform(xgbc_preds)

In [682]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [683]:
print(accuracy_score(y_test, xgbc_preds))

0.6


In [684]:
print(confusion_matrix(y_test, xgbc_preds))

[[1 0 0]
 [1 2 0]
 [0 1 0]]


In [685]:
print(classification_report(y_test, xgbc_preds))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.67      0.67      0.67         3
           2       0.00      0.00      0.00         1

    accuracy                           0.60         5
   macro avg       0.39      0.56      0.44         5
weighted avg       0.50      0.60      0.53         5



c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [686]:
xgbc_preds

array([0, 0, 1, 1, 1], dtype=int64)

### grid search cross validated + XGBoost Classifier - same rseult as initial one

In [688]:
from sklearn.model_selection import GridSearchCV

In [699]:
eval_set = [(X, y)] # validation set for early stopping

In [706]:
xgbc_model2 = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [707]:
param_grid = {
    "subsample":[0.75, 1],
     "colsample_bytree":[0.75, 1],
     "max_depth":[2, 6],
     "min_child_weight":[1, 5],
     "learning_rate":[0.1, 0.01],
}

In [711]:
xgbc_grid = GridSearchCV(
    xgbc_model2,
    param_grid,
    cv=3,
    scoring="neg_log_loss"
)

In [712]:
xgbc_grid.fit(X_train,y_train,eval_set=eval_set,verbose=1)

[0]	validation_0-mlogloss:1.08700
[1]	validation_0-mlogloss:1.06847
[2]	validation_0-mlogloss:1.05218
[3]	validation_0-mlogloss:1.02871
[4]	validation_0-mlogloss:1.00754
[5]	validation_0-mlogloss:0.97829
[6]	validation_0-mlogloss:0.97301
[7]	validation_0-mlogloss:0.96943
[8]	validation_0-mlogloss:0.95028
[9]	validation_0-mlogloss:0.93418
[10]	validation_0-mlogloss:0.92190
[11]	validation_0-mlogloss:0.91078
[12]	validation_0-mlogloss:0.90091
[13]	validation_0-mlogloss:0.89272
[14]	validation_0-mlogloss:0.87871
[15]	validation_0-mlogloss:0.87405
[16]	validation_0-mlogloss:0.85960
[17]	validation_0-mlogloss:0.84922
[18]	validation_0-mlogloss:0.84631
[19]	validation_0-mlogloss:0.84320
[20]	validation_0-mlogloss:0.84041
[21]	validation_0-mlogloss:0.83951
[22]	validation_0-mlogloss:0.82438


[23]	validation_0-mlogloss:0.81991
[24]	validation_0-mlogloss:0.80815
[25]	validation_0-mlogloss:0.80566
[26]	validation_0-mlogloss:0.80021
[27]	validation_0-mlogloss:0.79883
[28]	validation_0-mlogloss:0.78868
[29]	validation_0-mlogloss:0.78795
[30]	validation_0-mlogloss:0.78279
[31]	validation_0-mlogloss:0.78282
[32]	validation_0-mlogloss:0.77236
[33]	validation_0-mlogloss:0.76792
[34]	validation_0-mlogloss:0.76120
[35]	validation_0-mlogloss:0.75430
[36]	validation_0-mlogloss:0.74607
[37]	validation_0-mlogloss:0.74549
[38]	validation_0-mlogloss:0.74271
[39]	validation_0-mlogloss:0.74411
[40]	validation_0-mlogloss:0.74214
[41]	validation_0-mlogloss:0.73997
[42]	validation_0-mlogloss:0.73813
[43]	validation_0-mlogloss:0.73689
[44]	validation_0-mlogloss:0.73635
[45]	validation_0-mlogloss:0.73110
[46]	validation_0-mlogloss:0.73169
[47]	validation_0-mlogloss:0.73210
[48]	validation_0-mlogloss:0.72691
[49]	validation_0-mlogloss:0.72901
[50]	validation_0-mlogloss:0.72257
[51]	validation_0-ml

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[70]	validation_0-mlogloss:0.74280
[71]	validation_0-mlogloss:0.74396
[72]	validation_0-mlogloss:0.74361
[73]	validation_0-mlogloss:0.74325
[74]	validation_0-mlogloss:0.74657
[75]	validation_0-mlogloss:0.75132
[76]	validation_0-mlogloss:0.74908
[77]	validation_0-mlogloss:0.75020
[78]	validation_0-mlogloss:0.75333
[79]	validation_0-mlogloss:0.75763
[80]	validation_0-mlogloss:0.75474
[81]	validation_0-mlogloss:0.75586
[82]	validation_0-mlogloss:0.75675
[83]	validation_0-mlogloss:0.75905
[84]	validation_0-mlogloss:0.76100
[85]	validation_0-mlogloss:0.76292
[86]	validation_0-mlogloss:0.76396
[87]	validation_0-mlogloss:0.76071
[88]	validation_0-mlogloss:0.76135
[89]	validation_0-mlogloss:0.76386
[90]	validation_0-mlogloss:0.76689
[91]	validation_0-mlogloss:0.76819
[92]	validation_0-mlogloss:0.76602
[93]	validation_0-mlogloss:0.76649
[94]	validation_0-mlogloss:0.76520
[95]	validation_0-mlogloss:0.76219
[96]	validation_0-mlogloss:0.76546
[97]	validation_0-mlogloss:0.76964
[98]	validation_0-ml

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

...

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[86]	validation_0-mlogloss:1.00602
[87]	validation_0-mlogloss:1.00569
[88]	validation_0-mlogloss:1.00537
[89]	validation_0-mlogloss:1.00505
[90]	validation_0-mlogloss:1.00472
[91]	validation_0-mlogloss:1.00440
[92]	validation_0-mlogloss:1.00409
[93]	validation_0-mlogloss:1.00377
[94]	validation_0-mlogloss:1.00345
[95]	validation_0-mlogloss:1.00314
[96]	validation_0-mlogloss:1.00282
[97]	validation_0-mlogloss:1.00251
[98]	validation_0-mlogloss:1.00220
[99]	validation_0-mlogloss:1.00189
[0]	validation_0-mlogloss:1.06366
[1]	validation_0-mlogloss:1.02023
[2]	validation_0-mlogloss:0.98850
[3]	validation_0-mlogloss:0.95554
[4]	validation_0-mlogloss:0.91556
[5]	validation_0-mlogloss:0.87691
[6]	validation_0-mlogloss:0.83912
[7]	validation_0-mlogloss:0.82128
[8]	validation_0-mlogloss:0.80151
[9]	validation_0-mlogloss:0.78069
[10]	validation_0-mlogloss:0.75425
[11]	validation_0-mlogloss:0.73588
[12]	validation_0-mlogloss:0.71853
[13]	validation_0-mlogloss:0.69811
[14]	validation_0-mlogloss:0.6

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[72]	validation_0-mlogloss:0.41091
[73]	validation_0-mlogloss:0.41113
[74]	validation_0-mlogloss:0.40902
[75]	validation_0-mlogloss:0.40810
[76]	validation_0-mlogloss:0.40720
[77]	validation_0-mlogloss:0.40731
[78]	validation_0-mlogloss:0.40554
[79]	validation_0-mlogloss:0.40552
[80]	validation_0-mlogloss:0.40498
[81]	validation_0-mlogloss:0.40416
[82]	validation_0-mlogloss:0.40492
[83]	validation_0-mlogloss:0.40344
[84]	validation_0-mlogloss:0.40402
[85]	validation_0-mlogloss:0.40222
[86]	validation_0-mlogloss:0.40281
[87]	validation_0-mlogloss:0.40488
[88]	validation_0-mlogloss:0.40556
[89]	validation_0-mlogloss:0.40714
[90]	validation_0-mlogloss:0.40853
[91]	validation_0-mlogloss:0.40776
[92]	validation_0-mlogloss:0.40833
[93]	validation_0-mlogloss:0.40892
[94]	validation_0-mlogloss:0.40915
[95]	validation_0-mlogloss:0.40716
[96]	validation_0-mlogloss:0.40775
[97]	validation_0-mlogloss:0.40782
[98]	validation_0-mlogloss:0.40759
[99]	validation_0-mlogloss:0.40713


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constrain...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=3, ...),
             param_grid={'colsample_bytree': [0.75, 1],
                         'learning_rate': [0.1, 0.01], 'max_depth': [2, 6],
                         'min_child_weight': [1, 5], 'subsample': [0.75, 1]},
             scoring='neg_log_loss')

In [713]:
xgbc_grid.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 0.1,
 'max_depth': 2,
 'min_child_weight': 1,
 'subsample': 0.75}

In [716]:
xgbc_cv_best = XGBClassifier(
    colsample_bytree = 0.75,
    learning_rate = 0.1,
    max_depth = 2,
    min_child_weight = 1,
    subsample = 0.75,
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [717]:
xgbc_cv_best.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3, ...)

In [719]:
xgbc_cv_best_preds = xgbc_cv_best.predict(X_test)
xgbc_cv_best_preds = encoder.inverse_transform(xgbc_cv_best_preds)

In [720]:
confusion_matrix(y_test, xgbc_cv_best_preds)

array([[1, 0, 0],
       [1, 2, 0],
       [0, 1, 0]], dtype=int64)

In [721]:
print(classification_report(y_test, xgbc_cv_best_preds))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.67      0.67      0.67         3
           2       0.00      0.00      0.00         1

    accuracy                           0.60         5
   macro avg       0.39      0.56      0.44         5
weighted avg       0.50      0.60      0.53         5



c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
